In [1]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import json

In [2]:
client = MongoClient('localhost', 27017)
db = client.db
comments = db.comments

In [3]:
commentdf = pd.DataFrame(comments.find())

In [4]:
def string_to_list(string):
    return json.loads(string)

def get_len(list):
    return len(list)

In [5]:
commentdf['stocks'] = commentdf['stocks'].apply(string_to_list)

In [6]:
commentdf['size'] = commentdf['stocks'].apply(get_len)

In [7]:
stockdf = commentdf.loc[commentdf['size'] != 0]

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer
s = stockdf['stocks']
mlb = MultiLabelBinarizer()
onehotdf = pd.DataFrame(mlb.fit_transform(s), columns = mlb.classes_, index = stockdf.index)

connectionsdict = onehotdf.sum(axis = 1).to_dict()
connectionsdict = {k:v for k, v in connectionsdict.items() if v != 1}

onehotdf = onehotdf[onehotdf.index.isin(connectionsdict.keys())]

chorddict = onehotdf.sum(axis = 0).to_dict()
chorddict = {key:val for key, val in chorddict.items() if val > 0}

nonMatrix = ['NaN', 'ACB', 'ALL', 'AM', 'AN', 'ANY', 'AONE', 'APT', 'AT', 'BB', 'BE', 'BEST', 'BIG', 'BILL', 'BOX', 'BRO', 'BY', 'CEO', 'DNOW', 'ET', 'EVER', 'FDS', 'FL', 'FOR', 'FREE', 'GDP', 'GL', 'HAS', 'HD', 'HE', 'HOME', 'HOPE', 'IQ', 'JP', 'KIM', 'LIFE', 'LIVE', 'LL', 'LOW', 'MAN', 'NEXT', 'NOW', 'OI', 'ONE', 'OUT', 'PLAN', 'PLAY', 'PM', 'PPT', 'PUMP', 'RE', 'REAL', 'RH', 'RTX', 'RUN', 'SAVE', 'SE', 'SGU', 'SO', 'SPXC', 'STAR          ', 'STAY', 'TDA', 'TEAM', 'TECH', 'TELL', 'TIMB$', 'TISI', 'TV', 'TWO', 'UI', 'TD', 'GLG']

take_or = lambda x, y: x if x >= y else y
onehotdf['BRK'] = onehotdf['BRK.A'].combine(onehotdf['BRK.B'], take_or)
onehotdf.drop(columns = ['BRK.A', 'BRK.B'], inplace = True)
onehotdf['GOOG'] = onehotdf['GOOG'].combine(onehotdf['GOOGL'], take_or)
onehotdf.drop(columns = ['GOOG', 'GOOGL'], inplace = True)

onehotdf.drop(columns = nonMatrix, inplace = True)

chorddict = onehotdf.sum(axis = 0).to_dict()
chorddict = {key:val for key, val in chorddict.items() if val > 100}

chorddf = onehotdf[chorddict.keys()]

chorddf = chorddf.T
testdict = chorddf.dot(chorddf.T).sum().sort_values().to_dict()

names = list(testdict.keys())[113:]
chorddf = onehotdf[names]

matrix = chorddf.corr()

matrix[matrix < 0] = 0
matrix[matrix == 1] = 0
matrix = matrix.values.tolist()

In [9]:
from chord import Chord
Chord(matrix, names).to_html()

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer
s = stockdf['stocks']
mlb = MultiLabelBinarizer()
onehotdf = pd.DataFrame(mlb.fit_transform(s), columns = mlb.classes_, index = stockdf.index)

In [11]:
connectionsdict = onehotdf.sum(axis = 1).to_dict()
connectionsdict = {k:v for k, v in connectionsdict.items() if v != 1}

onehotdf = onehotdf[onehotdf.index.isin(connectionsdict.keys())]

In [13]:
companydict = onehotdf.sum(axis = 0).to_dict()
companydict = {k:v for k, v in companydict.items() if v == 0}

onehotdf.drop(columns = companydict.keys(), inplace = True)

In [15]:
onehotdf = onehotdf.T

In [16]:
onehotdf

,52,56,67,73,91,157,323,377,387,403,...,2392139,2392197,2392242,2392254,2392328,2392344,2392413,2392510,2392539,2392573
A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AACQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AACQU,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAL,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZUMZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZUO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZVO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
from scipy import sparse
import networkx as nx

matrix = onehotdf.to_numpy()
sparsematrix = sparse.csr_matrix(matrix)

adjacencyframe = pd.DataFrame(sparsematrix.dot(onehotdf.T), columns = onehotdf.index)
adjacencyframe.set_index(onehotdf.index, inplace= True)

In [29]:
G = nx.from_pandas_adjacency(adjacencyframe)

In [30]:
pos = nx.spring_layout(G)

In [33]:
import plotly.offline as py
import plotly.graph_objects as go

In [37]:
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 10,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))

for node in G.nodes():
    x, y = pos[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])

In [38]:
def make_edge(x, y, text, width):
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')

In [55]:
edge_trace = []
for edge in G.edges():
    
    if G.edges()[edge]['weight'] > 0:
        char_1 = edge[0]
        char_2 = edge[1]
    x0, y0 = pos[char_1]
    x1, y1 = pos[char_2]
    text   = 'test'
        
    trace  = make_edge([x0, x1, None], [y0, y1, None], text, 
                    width = 0.3*G.edges()[edge]['weight']**1.75)
    edge_trace.append(trace)

In [57]:
# Customize layout
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # transparent background
    plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)
# Create figure
fig = go.Figure(layout = layout)
# Add all edge traces
for trace in edge_trace:
    fig.add_trace(trace)
# Add node trace
fig.add_trace(node_trace)
# Remove legend
fig.update_layout(showlegend = False)
# Remove tick labels
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)
# Show figure
fig.write_html('adjacency.html')